In [3]:
import numpy as np
import pandas as pd
import sklearn
import seaborn as sns
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from imblearn.over_sampling import RandomOverSampler, BorderlineSMOTE 

In [4]:
df=pd.read_csv('train_input.csv')

In [5]:
df_copy = df.copy()

In [6]:
lastcolumn_np = df.iloc[:,-1:].to_numpy()
lastcolumn_op=np.ravel(lastcolumn_np)

In [7]:
df_copy.drop(['Target Variable (Discrete)','Feature 16','Feature 17'], axis=1, inplace=True)

In [8]:
imp_mean = IterativeImputer(random_state=0)
imp_mean.fit(df_copy)
IterativeImputer(random_state=0)
data = imp_mean.transform(df_copy)

In [56]:
correlated_features = set()
correlation_matrix = pd.DataFrame(data).corr()
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)

{10, 20, 12, 13}


In [57]:
np_drop = np.delete(data, [10,20,12,13],1)

In [58]:
Dict = {17:10, 12:10, 11:10, 16:10, 10:10, 9:10}

In [59]:
over = RandomOverSampler(random_state=0, sampling_strategy=Dict)

In [60]:
X_resampled, y_resampled = over.fit_resample(np_drop, lastcolumn_op)

In [61]:
sm = BorderlineSMOTE(random_state=0, k_neighbors=2, kind='borderline-2')

In [62]:
X_resampled, y_resampled = sm.fit_resample(X_resampled, y_resampled)

In [49]:
n_estimators = [2000]
max_features = ['auto', 'sqrt']
max_depth = [50,60,70]
min_samples_split = [2, 5]
min_samples_leaf = [1, 2]
bootstrap = [True, False]# Create the random grid
params = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
cv_result = RandomizedSearchCV(RandomForestClassifier(),params,cv=3,scoring='accuracy', random_state=42)
cv_result.fit(X_resampled, y_resampled)
cv_result.best_params_

{'n_estimators': 2000,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 70,
 'bootstrap': False}

In [63]:
data_7 = RandomForestClassifier(n_estimators = 2000, min_samples_split=2, min_samples_leaf=1, max_features='auto',
                               max_depth=70, random_state = 42, bootstrap=False)
data_7.fit(X_resampled, y_resampled)

RandomForestClassifier(bootstrap=False, max_depth=70, n_estimators=2000,
                       random_state=42)

In [64]:
df_2=pd.read_csv('iith_foml_2020_test.csv')

In [65]:
df_2_copy = df_2.copy()
df_2_copy.drop(['Feature 16','Feature 17'], axis=1, inplace=True)

In [66]:
data_2 = imp_mean.transform(df_2_copy)

In [67]:
np_drop_2 = np.delete(data_2, [10,20,12,13],1)

In [68]:
data_pred = data_7.predict(np_drop_2)

In [78]:
col_id = np.linspace(1,426,426,dtype=int)

In [79]:
Final_df = pd.DataFrame(columns=['Id','Category'])

In [80]:
Final_df['Id'] = col_id

In [82]:
Final_df['Category'] = data_pred

In [87]:
Final_df.to_csv('test_ouput.csv', index=False)